In [ ]:
import tensorflow as tf
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mplfinance as mpf
import talib as talib
import numpy as np
import data as ds
import common as common
import os as os
import math as math
import datetime as datetime
from datetime import timedelta
from os import listdir, walk
from itertools import repeat
from mplfinance.original_flavor import candlestick_ohlc
from datetime import datetime, timedelta
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler, StandardScaler
pd.options.mode.chained_assignment = None

In [ ]:
# 股價
data_source = ds.DataSource()
codes1 = ['hsi', '02800', '03033']
data1 = data_source.get_data(codes1)
for code, df0 in data1.items():
    df0.fillna(0, inplace=True)
    df0.replace([np.inf, -np.inf], 0, inplace=True)
    df0.columns = ['code', 'lot', 'nmll', 'sdate', 'High', 'Low', 'Open', 'Close', 'Volume']
    types2 = {'sdate': 'object', 'High': 'float64', 'Low': 'float64', 'Open': 'float64', 'Close': 'float64', 'Volume': 'int64'}
    df0.astype(types2).dtypes
    data1[code] = df0

In [ ]:
df1 = data1['hsi']
df1.sdate = pd.to_datetime(df1.sdate)
df1.index = df1.sdate
df1['year'] = pd.DatetimeIndex(df1.sdate).year

In [ ]:
# 單日升/跌幅
df2 = df1.copy(deep=True)
df2['gap_last'] = df2['Close'].diff()
df2['gap_last_pct'] = (df2['gap_last'] / df2['Close'])*100
mask = ((df2['gap_last_pct'] >= 8) | (df2['gap_last_pct'] <= -8))
df2.loc[mask]

In [ ]:
# 開市裂口
df3 = df1.copy(deep=True)
df3['pre_last'] = df3['Close'].shift(periods=1, fill_value=0)
df3['gap_open'] = df3['Open']-df3['pre_last']
df3['gap_open_pct'] = (df3['gap_open'] / df3['Close'])*100
mask = (((df3['gap_open_pct'] >= 2) | (df3['gap_open_pct'] <= -100)) & (df3['year'] >= 2020))
days1 = df3.loc[mask].iloc[1:].sdate

In [ ]:
# 繪圖
for v in days1:
    shift_day = 30
    start, end = v-timedelta(days=shift_day), v+timedelta(days=shift_day)
    mask = (df3['sdate'] > start) & (df3['sdate'] <= end)
    df4 = df3.loc[mask]
    df4['Volume'] = df4['Volume']/100000000
    style = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size':6})
    scale_width_adjustment = dict(volume=0.6, candle=1)
    ylabel = df4['code'].values.tolist()[0]
    year = str(df4['year'].values.tolist()[0])
    vlines=dict(vlines=[v])
    mpf.plot(df4, type='candle', vlines=vlines, style=style, title='', ylabel=ylabel+'    '+year, ylabel_lower='', volume=True, figscale=0.5, xrotation=0, show_nontrading=False, tight_layout=True, datetime_format="%m-%d", scale_width_adjustment=scale_width_adjustment) 